In [1]:
from tuplestate import init_from_solvitaire, to_dict, pprint_st
from benchmarking import *
from solver import solve

In [2]:
with open('./bench/47-solvedmin.txt') as f:
    ret = f.read()

In [3]:
def get_state(ret):
    deck_json = convert_shootme_to_solvitaire_json(ret)
    return init_from_solvitaire(deck_json)

In [4]:
def prepare_ui_state(ret):
    parsed = parse_winnable(ret)
    game_state = to_dict(get_state(ret))
    
    def card(c):
        ''' UI uses 10 instead of T '''
        return c.replace('t', '10').replace('T', '10')
    
    return {
        'stock': [card(c).lower() for c in game_state['stock']],
        'tableau': [
            [card(c) for c in tab] 
            for tab in game_state['tableau']
        ],
        'waste': game_state['waste'],
        'foundation': game_state['foundation'],
        'moveSeq': parsed['moves'].strip().split(' ')
    }

In [5]:
print(prepare_ui_state(ret))

{'stock': ['4d', 'ah', 'ac', '9d', '8c', '8h', '6s', '3s', '5d', '4s', 'jd', '2c', '4h', 'ks', '6h', 'jh', '6d', '7s', '3c', 'kc', 'jc', '3d', '2d', '3h'], 'tableau': [['7H'], ['10s', '9H'], ['ad', 'qs', 'AS'], ['kd', '6c', '5s', 'JS'], ['7d', '8s', '5c', '2h', '10D'], ['8d', 'qd', '9s', '4c', 'qh', '2S'], ['qc', 'kh', '10c', '10h', '5h', '7c', '9C']], 'waste': [], 'foundation': [[], [], [], []], 'moveSeq': ['3S', 'F3', '6S', 'F6', 'DR3', 'W3', 'DR3', 'W1', 'WS', 'W1', 'WS', '46', 'F4', 'DR2', 'W4', 'WH', 'WC', '56', 'F5', '5H', 'F5', '45', '4S', 'F4', '14', '1S', '41-2', 'F4', '34-2', 'F3', '3D', 'NEW', 'DR2', 'W5', 'W3', '63-3', 'F6', 'DR2', 'WC', '5C', 'DR2', 'NEW', 'DR1', 'W6', 'WD', '6D', 'WH', '5D', '6C', 'F6', '5C', 'F5', '1D', 'DR1', 'WD', 'WS', 'DR1', 'WH', '5S', 'F5', '6S', 'F6', '5D', '1C', '73', 'F7', '7C', 'F7', '7H', 'F7', 'W5', 'WH', '1H', 'DR1', 'WC', 'WH', '3C', '2H', 'F2', '7H', 'F7', '7C', 'F7', '4H', '2S', '65', 'F6', '6D', 'DR1', 'WD', '3D', '3S', 'WD', 'WC', '3H',

In [6]:
solution = solve(get_state(ret))
solution

()